# Background
- Wanting to get some serious and structured documentation detailing what we've learned and showed with the Hall probe setup. And will build onto this as we get more data.
- As a refresher (have not looked at Hall probe data in awhile), starting with a simple exploration

# Imports

In [10]:
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from emtracks.plotting import config_plots
config_plots()

# Loading Files

## Get file names

In [59]:
datadir = "/home/ckampa/Dropbox/LogFiles/"

In [60]:
files = sorted(os.listdir(datadir))

In [61]:
len(files)

241

- Lots of files to parse...

## Break up into file type
- Let's try to split files based on generically how they are parsed (e.g. temp. enclosure logs are parsed much differently than slow controls logs)

In [8]:
files

['1811121_02-03-20_1341.txt',
 '1811121_02-19-20_1436.txt',
 '1811121_03-16-20_1226.txt',
 '1811121_06-18-20_1240.txt',
 '1811121_06-26-20_1046.txt',
 '1811121_07-02-19_1512.txt',
 '1811121_08-01-19_0820.txt',
 '1811121_11-08-19_1227 chamber.txt',
 '1811121_12-03-19_1622.txt',
 '1811121_12-03-19_1624 chamber.txt',
 '2018-11-19 134630.txt',
 '2018-11-19 135306.txt',
 '2018-11-19 135401.txt',
 '2018-11-29 113332.txt',
 '2018-11-29 113749.txt',
 '2018-11-29 123435.txt',
 '2018-11-29 133933.txt',
 '2018-11-29 135237.txt',
 '2018-12-03 113432.txt',
 '2018-12-03 115928.txt',
 '2018-12-03 123641.txt',
 '2018-12-03 130522.txt',
 '2018-12-03 152047.txt',
 '2018-12-03 153304.txt',
 '2018-12-03 183641 (For Micah and Alex).txt',
 '2018-12-04 121533.txt',
 '2018-12-04 190642.txt',
 '2018-12-05 203946.txt',
 '2018-12-06 061040.txt',
 '2018-12-06 140422.txt',
 '2018-12-06 152408.txt',
 '2018-12-06 172741.txt',
 '2018-12-07 085600.txt',
 '2018-12-07 094111.txt',
 '2018-12-07 094743.txt',
 '2018-12-07 

In [12]:
for file in files[:10]:
    with open(datadir+file, 'r') as f:
        print(f"Reading File: {file}")
        print("---------------------------------------")
        print(f.readlines(3))
        print("\n\n")

Reading File: 1811121_02-03-20_1341.txt
---------------------------------------
[]



Reading File: 1811121_02-19-20_1436.txt
---------------------------------------
['43880.608715 *Unit SN: 1811121\n']



Reading File: 1811121_03-16-20_1226.txt
---------------------------------------
['43906.518692 *Unit SN: 1811121\n']



Reading File: 1811121_06-18-20_1240.txt
---------------------------------------
['44000.528553 *Unit SN: 1811121\n']



Reading File: 1811121_06-26-20_1046.txt
---------------------------------------
['44008.449201 *Unit SN: 1811121\n']



Reading File: 1811121_07-02-19_1512.txt
---------------------------------------
['43648.633553 *Unit SN: 1811121\n']



Reading File: 1811121_08-01-19_0820.txt
---------------------------------------
['43678.347535 *Unit SN: 1811121\n']



Reading File: 1811121_11-08-19_1227 chamber.txt
---------------------------------------
['43777.519109 *Unit SN: 1811121\n']



Reading File: 1811121_12-03-19_1622.txt
--------------------------

In [13]:
len(files)

242

In [19]:
useful_files = []
for file in files:
    with open(datadir+file, 'r') as f:
        print(f"Reading File: {file}")
        print("---------------------------------------")
        print(f.readlines()[:15])
    useful = input("Useful file? ")
    print('\n')
    if useful == 'y':
        useful_files.append(useful)

Reading File: 1811121_02-03-20_1341.txt
---------------------------------------
[]
Useful file? n


Reading File: 1811121_02-19-20_1436.txt
---------------------------------------
['43880.608715 *Unit SN: 1811121\n', 'Smart Pad FV# 2042\n', 'ACE FV# 2047\n', 'Units: C\n', 'PGC SmartLog version 3.2.1\n', 'Temp Only Mode\n', 'air sp: 15.00\n', 'air pid: 03.00 / 05.00 / 000\n', 'wat sp: \n', 'wat pid:  /  / \n', 'rh sp: \n', 'rh pid:  /  / \n', 'aux sp: 13.00\n', 'aux pid: 05.00 / 01.00 / 000\n', 'aux ctrl: auto\n']
Useful file? n


Reading File: 1811121_03-16-20_1226.txt
---------------------------------------
['43906.518692 *Unit SN: 1811121\n', 'Smart Pad FV# 2042\n', 'ACE FV# 2047\n', 'Units: C\n', 'PGC SmartLog version 3.2.1\n', 'Temp Only Mode\n', 'air sp: 15.00\n', 'air pid: 03.00 / 05.00 / 000\n', 'wat sp: \n', 'wat pid:  /  / \n', 'rh sp: \n', 'rh pid:  /  / \n', 'aux sp: 13.00\n', 'aux pid: 05.00 / 01.00 / 000\n', 'aux ctrl: auto\n']
Useful file? n


Reading File: 1811121_06-18-

Useful file? y


Reading File: 2018-11-29 113332.txt
---------------------------------------
['11/29/2018 11:34:11 AM\n', 'Angle0, Angle1, Step0, Step1, Resolution0, Resolution1, N_Field, N_FFT, N_Hall_sensors, DF0000000D5E803A, 380000000D59A03A, 8E0000000D51483A, C90000000D53983A, BA0000000D60443A, AB0000000D57943A, 6E0000000D665A3A, FA0000000D60163A, 480000000D4DAE3A, 700000000D4F613A, 4C0000000D55C93A, 140000000D5FB53A, D50000000D60C33A, 6A0000000D61333A, C80000000D554B3A, C50000000D5E473A, 2F0000000D5EC73A, 480000000D5D273A, 8F0000000D5B373A, 870000000D4E8F3A\n', '-5.000000000,-5000000.000000000,0.526316000,0.000000526,20.000000000,20.000000000,250.000000000,400.000000000,20.000000000,0.000000000,1.000000000,2.000000000,3.000000000,4.000000000,5.000000000,6.000000000,7.000000000,8.000000000,9.000000000,10.000000000,11.000000000,12.000000000,13.000000000,14.000000000,15.000000000,16.000000000,17.000000000,18.000000000,19.000000000\n', '\n', '11/29/2018 11:34:40 AM\n', 'SmarAct_Patte

KeyboardInterrupt: 

In [23]:
files[0][:7]

'1811121'

In [49]:
enclosure_logs = []
config_logs = []
slow_control_logs = []
hall_only_logs = []
calibration_files = []
etc_logs = []
for file in files:
    if file[:7] == '1811121':
        enclosure_logs.append(file)
    elif ('config' in file):
        config_logs.append(file)
    elif ('slow' in file) | (file[:4] == '2018'):
        slow_control_logs.append(file)
    elif ('hallonly' in file) | (file[:4] == '2019') | (file[:4] == '2020'):
        hall_only_logs.append(file)
    elif ('cal' in file) & ('.dat' in file):
        calibration_files.append(file)
    else:
        etc_logs.append(file)

In [50]:
len(enclosure_logs), len(config_logs), len(slow_control_logs), len(hall_only_logs), len(calibration_files), len(etc_logs)

(10, 18, 172, 34, 2, 6)

In [51]:
np.sum([len(enclosure_logs), len(config_logs), len(slow_control_logs), len(hall_only_logs), len(calibration_files), len(etc_logs)])

242

In [52]:
len(files)

242

In [53]:
enclosure_logs

['1811121_02-03-20_1341.txt',
 '1811121_02-19-20_1436.txt',
 '1811121_03-16-20_1226.txt',
 '1811121_06-18-20_1240.txt',
 '1811121_06-26-20_1046.txt',
 '1811121_07-02-19_1512.txt',
 '1811121_08-01-19_0820.txt',
 '1811121_11-08-19_1227 chamber.txt',
 '1811121_12-03-19_1622.txt',
 '1811121_12-03-19_1624 chamber.txt']

In [54]:
config_logs

['2019-04-02 132831config.txt',
 '2019-04-04 125509config.txt',
 '2019-04-04 203855config.txt',
 '2019-04-05 202534config.txt',
 '2019-04-06 132816config.txt',
 '2019-04-07 101100config.txt',
 '2019-04-15 094402config.txt',
 '2019-04-17 083615config.txt',
 '2019-04-27 160309config.txt',
 '2019-05-16 102521config.txt',
 '2019-05-21 131236config.txt',
 '2019-05-24 074600config.txt',
 '2019-05-28 150507config.txt',
 '2019-05-29 130502config.txt',
 '2019-06-03 083417config.txt',
 '2019-06-05 143924config.txt',
 '2019-06-06 133006config.txt',
 '2019-06-06 133231config.txt']

In [55]:
slow_control_logs

['2018-11-19 134630.txt',
 '2018-11-19 135306.txt',
 '2018-11-19 135401.txt',
 '2018-11-29 113332.txt',
 '2018-11-29 113749.txt',
 '2018-11-29 123435.txt',
 '2018-11-29 133933.txt',
 '2018-11-29 135237.txt',
 '2018-12-03 113432.txt',
 '2018-12-03 115928.txt',
 '2018-12-03 123641.txt',
 '2018-12-03 130522.txt',
 '2018-12-03 152047.txt',
 '2018-12-03 153304.txt',
 '2018-12-03 183641 (For Micah and Alex).txt',
 '2018-12-04 121533.txt',
 '2018-12-04 190642.txt',
 '2018-12-05 203946.txt',
 '2018-12-06 061040.txt',
 '2018-12-06 140422.txt',
 '2018-12-06 152408.txt',
 '2018-12-06 172741.txt',
 '2018-12-07 085600.txt',
 '2018-12-07 094111.txt',
 '2018-12-07 094743.txt',
 '2018-12-07 100019.txt',
 '2018-12-07 100549.txt',
 '2018-12-07 132225.txt',
 '2018-12-07 143730.txt',
 '2018-12-07 145759.txt',
 '2018-12-07 152901.txt',
 '2018-12-07 165201.txt',
 '2018-12-07 201507.txt',
 '2018-12-07 205045.txt',
 '2018-12-10 131807.txt',
 '2018-12-10 132054.txt',
 '2018-12-10 142131.txt',
 '2018-12-10 1505

In [56]:
hall_only_logs

['2019-02-28 151111.txt',
 '2019-04-02 132746.txt',
 '2019-04-04 125330.txt',
 '2019-04-04 203754.txt',
 '2019-04-04 210509.txt',
 '2019-04-05 202523.txt',
 '2019-04-06 132802.txt',
 '2019-04-07 100858.txt',
 '2019-04-15 094252.txt',
 '2019-04-17 083608.txt',
 '2019-04-27 160216.txt',
 '2019-05-16 102245.txt',
 '2019-05-16 102515.txt',
 '2019-05-21 131233.txt',
 '2019-05-24 074557.txt',
 '2019-05-28 150504.txt',
 '2019-05-29 130459.txt',
 '2019-06-03 083346.txt',
 '2019-06-05 143920.txt',
 '2019-06-06 133003.txt',
 '2019-06-06 133226.txt',
 '2019-08-13 1155830 hallonly.txt',
 '2019-08-15 122400.txt',
 '2019-08-19 092500.txt',
 '2019-08-23 162500.txt',
 '2019-08-23 182500.txt',
 '2019-08-27 094500.txt',
 '2019-08-27 101000.txt',
 '2019-08-27 102600.txt',
 '2019-08-27 132800.txt',
 '2019-08-30 122400.txt',
 '2019-08-30 122800.txt',
 '2019-08-30 123800.txt',
 '2019-09-04 080000.txt']

In [57]:
calibration_files

['caldat.dat', 'caldat12.dat']

In [58]:
etc_logs

['Micah Alex, 2nd try.txt',
 'Micah Alex, 2nd try.xlsx',
 'Micah Alex, no line 10.txt',
 'ZaberHigherLimit.lvm',
 'ZaberLowerLimit.lvm',
 'test.txt']

In [26]:
for file in files:
    if "chamber" in file:
        print(file)

1811121_11-08-19_1227 chamber.txt
1811121_12-03-19_1624 chamber.txt
